In [8]:
import bagpy
from bagpy import bagreader
import numpy as np
import pandas as pd
import rosbag
import os
from PIL import Image
import io


In [4]:
def read_bag_file(file_path, name):
    # Read the bag file
    bag = bagreader(file_path)
    
    # Get all topics in the bag file
    topics = bag.topics
    print(f"Topics in {name}:")
    
    for topic in topics:
        # Get the CSV file path for each topic
        topic_csv = bag.message_by_topic(topic)
        
        if topic_csv:
            print(f"  - {topic}: {topic_csv}")
        else:
            print(f"  - {topic}: No data found")

# Call the function for each bag file
read_bag_file('./Data/ROSBAG/Ch2_002/HMB_1.bag', 'HMB_1')
read_bag_file('./Data/ROSBAG/Ch2_002/HMB_2.bag', 'HMB_2')
read_bag_file('./Data/ROSBAG/Ch2_002/HMB_4.bag', 'HMB_4')
read_bag_file('./Data/ROSBAG/Ch2_002/HMB_5.bag', 'HMB_5')
read_bag_file('./Data/ROSBAG/Ch2_002/HMB_6.bag', 'HMB_6')

[INFO]  Data folder ./Data/ROSBAG/Ch2_002/HMB_1 already exists. Not creating.
Topics in HMB_1:
  - /can_bus_dbw/can_rx: ./Data/ROSBAG/Ch2_002/HMB_1/can_bus_dbw-can_rx.csv
  - /center_camera/camera_info: ./Data/ROSBAG/Ch2_002/HMB_1/center_camera-camera_info.csv
  - /center_camera/image_color/compressed: ./Data/ROSBAG/Ch2_002/HMB_1/center_camera-image_color-compressed.csv
  - /diagnostics: ./Data/ROSBAG/Ch2_002/HMB_1/diagnostics.csv
  - /ecef/: ./Data/ROSBAG/Ch2_002/HMB_1/ecef-.csv
  - /fix: ./Data/ROSBAG/Ch2_002/HMB_1/fix.csv
  - /imu/data: ./Data/ROSBAG/Ch2_002/HMB_1/imu-data.csv
  - /left_camera/camera_info: ./Data/ROSBAG/Ch2_002/HMB_1/left_camera-camera_info.csv
  - /left_camera/image_color/compressed: ./Data/ROSBAG/Ch2_002/HMB_1/left_camera-image_color-compressed.csv
  - /pressure: ./Data/ROSBAG/Ch2_002/HMB_1/pressure.csv
  - /right_camera/camera_info: ./Data/ROSBAG/Ch2_002/HMB_1/right_camera-camera_info.csv
  - /right_camera/image_color/compressed: ./Data/ROSBAG/Ch2_002/HMB_1/right

In [9]:
def extract_images(bag_file, output_dir, image_topic):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rosbag.Bag(bag_file, 'r') as bag:
        for topic, msg, t in bag.read_messages(topics=[image_topic]):
            if msg._type == "sensor_msgs/CompressedImage":
                # For compressed images
                img_np = np.frombuffer(msg.data, np.uint8)
                img = Image.open(io.BytesIO(img_np))
            elif msg._type == "sensor_msgs/Image":
                # For raw images
                img_np = np.frombuffer(msg.data, dtype=np.uint8).reshape(msg.height, msg.width, -1)
                img = Image.fromarray(img_np)
            else:
                print(f"Unsupported image type: {msg._type}")
                continue

            # Save the image
            img.save(os.path.join(output_dir, f"frame_{t.to_nsec()}.png"))

extract_images('./Data/ROSBAG/Ch2_002/HMB_1.bag', './Data/ROSBAG/Ch2_002/images/HMB_1','/left_camera/image_color/compressed')
extract_images('./Data/ROSBAG/Ch2_002/HMB_1.bag', './Data/ROSBAG/Ch2_002/images/HMB_1','/center_camera/image_color/compressed')
extract_images('./Data/ROSBAG/Ch2_002/HMB_1.bag', './Data/ROSBAG/Ch2_002/images/HMB_1','/right_camera/image_color/compressed')

KeyboardInterrupt: 